In [5]:
from matplotlib import pyplot as plt
%matplotlib notebook

import sys 
sys.path.append('..')

from ParserQuery import *
from Index import Index
from Weighter import WeighterVector, WeighterSchema3
from IRModel import Vectoriel
from Evaluation import *
from EvalIRModel import *

from diversity.Clustering import ClusteringDiversifier

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
index = Index('easyCLEF08', '../easyCLEF08/easyCLEF08_text.txt', index_path='../indexes/easyCLEF08/')
#index.indexation()
weighter2 = WeighterVector(index)
#weighter2.calculeNorms()
weighter3 = WeighterSchema3(index)
#weighter3.calculeNorms()
irm2 = Vectoriel(weighter2)
irm3 = Vectoriel(weighter3)

In [ ]:
qp = QueryParser()
qp.initFile('../easyCLEF08/easyCLEF08_query.txt', '../easyCLEF08/easyCLEF08_gt.txt')
qu = qp.nextQuery()
print(qu.text_, '\n', qu.relevants_)
prec = PrecisionAtN()
prec_cluster = ClusterRecallAtN()
rank = irm2.getRanking(qu.text_)

irlist = IRList(qu, rank)
print(prec.eval(ir_list=irlist))
print(prec_cluster.eval(irlist))

In [18]:
cd = ClusteringDiversifier(index, cluster_order_by='size_descending')

data, cluster_order, div = cd.diversify(irlist, by_top_n=30, n_clusters=6)
print(cluster_order)
print(div)
data

[2 1 3 4 0 5]
[21420, 35878, 36043, 15763, 35621, 4898, 32866, 35880, 6636, 23959, 15179, 6246, 37105, 37069, 8539, 10651, 36072, 4945, 36023, 37070, 11512, 22874, 36071, 37072, 8054, 37106, 37158, 37101, 37159, 37068]


,rank,cluster
id,,
21420,12,2
35878,1,1
36043,3,3
15763,15,4
35621,9,0
4898,7,5
32866,13,2
35880,2,1
6636,22,3
